In [1]:
#Import dependenices
import csv
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.model_selection import train_test_split

# Split the Data into Training and Testing Sets

In [2]:
# Specify the path to your CSV file
loan_path = '../Resources/Clean_test_scaled.csv'

# Read the CSV file into a DataFrame
loan_df = pd.read_csv(loan_path)

# Display the DataFrame
loan_df.head(5)


,Current loan amount,Credit score,Annual income,Monthly debt,Years of credit history,Number of open accounts,Number of credit problems,Current credit balance,Maximum open credit,Long term,Short term,Home mortgage,Own home,Rent,Bankrupt
0,-0.396710,-0.224366,0.810185,1.930978,0.519773,-0.416087,-0.332426,0.868213,0.402694,0,1,1,0,0,0
1,-0.406640,-0.233156,0.631936,1.489873,0.174878,-0.016795,-0.332426,1.019424,0.247734,0,1,1,0,0,0
2,-0.409901,-0.250059,-0.573923,-0.826455,-0.816695,-0.216441,-0.332426,-0.661535,-0.270115,0,1,0,0,1,0
3,-0.409245,-0.237888,-0.678656,-0.138913,-0.241870,0.981434,1.676190,-0.351042,-0.126429,0,1,0,0,1,1
4,-0.405046,-0.226394,0.450102,1.724058,1.123338,0.582142,-0.332426,0.180813,-0.189855,0,1,1,0,0,0


In [3]:
loan_df.shape

(8000, 15)

In [4]:
# Remove target from features data
y = loan_df.Bankrupt.values
X = loan_df.drop(columns='Bankrupt').values


In [5]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)


# Create a Neural Network Model

In [6]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=14))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 15        
                                                                 
 dense_1 (Dense)             (None, 1)                 2         
                                                                 
Total params: 17 (68.00 Byte)
Trainable params: 17 (68.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# Compile the Sequential model together and customise metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:

# Fit the model to the training data
fit_model = nn_model.fit(X_train, y_train, epochs=50)

Epoch 1/50


188/188 [==============================] - 8s 8ms/step - loss: 0.6609 - accuracy: 0.8905
Epoch 2/50
188/188 [==============================] - 2s 8ms/step - loss: 0.5991 - accuracy: 0.8917
Epoch 3/50
188/188 [==============================] - 2s 8ms/step - loss: 0.5484 - accuracy: 0.8917
Epoch 4/50
188/188 [==============================] - 2s 9ms/step - loss: 0.5069 - accuracy: 0.8917
Epoch 5/50
188/188 [==============================] - 2s 9ms/step - loss: 0.4733 - accuracy: 0.8917
Epoch 6/50
188/188 [==============================] - 2s 9ms/step - loss: 0.4460 - accuracy: 0.8917
Epoch 7/50
188/188 [==============================] - 1s 8ms/step - loss: 0.4240 - accuracy: 0.8917
Epoch 8/50
188/188 [==============================] - 2s 9ms/step - loss: 0.4062 - accuracy: 0.8917
Epoch 9/50
188/188 [==============================] - 1s 8ms/step - loss: 0.3921 - accuracy: 0.8917
Epoch 10/50
188/188 [==============================] - 2s 9ms/step - loss: 0.3808 - accuracy: 0.89

# Auto Optimisation

In [9]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=14))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=5,
            max_value=50,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [10]:

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2) 

# Run the kerastuner search for best hyperparameters
tuner.search(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

Reloading Tuner from .\untitled_project\tuner0.json


In [11]:

# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 3,
 'units_0': 40,
 'units_1': 45,
 'units_2': 10,
 'units_3': 45,
 'units_4': 15,
 'units_5': 45,
 'units_6': 10,
 'units_7': 50,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 3,
 'tuner/round': 0}

In [12]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

63/63 - 2s - loss: nan - accuracy: 0.8915 - 2s/epoch - 33ms/step
Loss: nan, Accuracy: 0.8914999961853027
